In [21]:
import yfinance as yf
import talib
import pandas as pd
from talib.abstract import *
import datetime as dt
from dateutil.parser import parse
from datetime import datetime, timezone
import os
import glob
from sklearn import datasets, linear_model
from datetime import datetime, timedelta
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import time


import pandas_datareader.data as dr
import matplotlib
import matplotlib.pyplot as plt

In [18]:
def reformat_multi_index_df(df):
    df_new = df.rename(columns = {'Close':'close', 'Open': 'open', 'High': 'high', 'Low': 'low', 'Volume':'volume'}, level=1 ) 
    df_new.fillna(method='ffill', inplace=True)
    df_new.fillna(method='bfill', inplace=True)

    #df_new.index=df.index.tz_localize(None)

    df_new.index = df_new.index.map(lambda t: t if isinstance(t, str) else t.strftime('%Y-%m-%d %H:%M'))
    df_new=df_new.groupby(df_new.index).first()
    #df_new.drop_duplicates(subset='index', inplace=True)
    return df_new

In [19]:
## IN SAMPLE
pepsi_coke_df = yf.download('PEP KO', auto_adjust=True, period='1d', start= "2020-01-01", end = "2020-05-30", group_by = 'ticker')
pepsi_coke_df = reformat_multi_index_df(pepsi_coke_df)

[*********************100%***********************]  2 of 2 completed


In [13]:
pepsi_df = pepsi_coke_df["PEP"]
coke_df = pepsi_coke_df["KO"]

In [14]:
pepsi_df.head()

,open,high,low,close,volume
Date,,,,,
2020-01-02 00:00,133.941754,134.294053,132.248771,132.914230,3784100
2020-01-03 00:00,132.561941,133.863473,132.238999,132.728302,4000100
2020-01-06 00:00,132.405346,133.403528,132.307480,133.237152,4085100
2020-01-07 00:00,133.090375,133.158885,131.084230,131.142944,5718100
2020-01-08 00:00,131.583336,132.464075,131.201680,131.818192,3681400


In [23]:
start_date = '2013-01-01'
end_date = '2020-02-29'

assets = ["PEP", "KO"]
datasets = dr.DataReader(assets, data_source='yahoo', start = start_date, end = end_date)['Adj Close']
datasets.head()

Symbols,PEP,KO
Date,,
2013-01-02,55.359879,29.618193
2013-01-03,55.383846,29.618193
2013-01-04,55.463696,29.665461
2013-01-07,55.455719,29.381893
2013-01-08,55.623398,29.177088


In [15]:
## take advantage of technical analysis
## boillinger bands gives the upper and lower std
bbands= BBANDS(pepsi_df)
ema = EMA(pepsi_df)

In [16]:
print(bbands)

                   upperband  middleband   lowerband
Date                                                
2020-01-02 00:00         NaN         NaN         NaN
2020-01-03 00:00         NaN         NaN         NaN
2020-01-06 00:00         NaN         NaN         NaN
2020-01-07 00:00         NaN         NaN         NaN
2020-01-08 00:00  133.913909  132.368164  130.822419
...                      ...         ...         ...
2020-05-22 00:00  133.670208  129.890533  126.110859
2020-05-26 00:00  130.164753  128.753596  127.342440
2020-05-27 00:00  129.585004  128.572322  127.559640
2020-05-28 00:00  130.477356  128.775276  127.073196
2020-05-29 00:00  130.764475  129.039317  127.314160

[103 rows x 3 columns]


In [40]:
print(ema)

Date
2020-01-02 00:00           NaN
2020-01-03 00:00           NaN
2020-01-06 00:00           NaN
2020-01-07 00:00           NaN
2020-01-08 00:00           NaN
                       ...    
2020-05-22 00:00    129.967650
2020-05-26 00:00    129.829830
2020-05-27 00:00    129.768278
2020-05-28 00:00    129.804769
2020-05-29 00:00    129.791870
Length: 103, dtype: float64


In [42]:
## can take advantage of pairs trading by trading spread
pepsi_mean = pepsi_df["close"].mean()
coke_mean = coke_df["close"].mean()

#pepsi_coke_spread = pepsi_mean/coke_mean

In [43]:

# when pepsi is undervalued compared to spread, sell or else buy
print(pepsi_coke_spread)

2.627845332778981
